<a href="https://colab.research.google.com/github/dkanzariya/Machine-learning-in-Communication-/blob/master/TP_communication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import math
from torch import nn
from torch.autograd import Variable

#An Introduction to deep learning for the physical layer

https://arxiv.org/pdf/1702.00832.pdf

In [2]:
class RadioTransNets(nn.Module):
  def __init__(self,in_channel,compossed_dim):
    
    super(RadioTransNets,self).__init__()
    self.in_channel = in_channel

    self.encoder = nn.Sequential(
        nn.Linear(in_channel , in_channel),
        nn.ReLU(inplace = True),
        nn.Linear(in_channel,compossed_dim))
    
    self.decoder = nn.Sequential(
        nn.Linear(compossed_dim , compossed_dim),
        nn.ReLU(inplace = True),
        nn.Linear(compossed_dim ,in_channel))

  def forward(self,x):
    x = self.encoder(x)

    x = (self.in_channel ** 2) * (x/x.norm(dim = -1)[: , None])

    noise = Variable(torch.randn(*x.size()) / ((2 * train_snr) ** 0.5))

    x+=noise
    x = self.decoder(x)

    return x

In [3]:
channel_size = 4
train_snr = 5.01187

train_labels = (torch.rand(100000) * channel_size).long()
train_data = torch.sparse.torch.eye(channel_size).index_select(dim=0, index=train_labels)

test_labels = (torch.rand(1500) * channel_size).long()
test_data = torch.sparse.torch.eye(channel_size).index_select(dim=0, index=test_labels)

### Model 

In [4]:
model = RadioTransNets(channel_size, compossed_dim=int(math.log2(channel_size)))
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()

def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.normal_(m.weight, std=0.01)

model.apply(init_weights)

RadioTransNets(
  (encoder): Sequential(
    (0): Linear(in_features=4, out_features=4, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=4, out_features=2, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=2, out_features=2, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=2, out_features=4, bias=True)
  )
)

In [5]:
for t in range(1000):

  train_data = Variable(train_data)
  y_train = model(train_data)
  loss = loss_fn(y_train, train_labels)
  if t%100 == 99:
    print("epoch : {}      loss : {}".format(t, loss.item()))

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

epoch : 99      loss : 1.3815325498580933
epoch : 199      loss : 0.9020938873291016
epoch : 299      loss : 0.8471445441246033
epoch : 399      loss : 0.8203009963035583
epoch : 499      loss : 0.8009700775146484
epoch : 599      loss : 0.7696080207824707
epoch : 699      loss : 0.4712601602077484
epoch : 799      loss : 0.4316464066505432
epoch : 899      loss : 0.415547251701355
epoch : 999      loss : 0.40644922852516174


In [9]:
y_test = model(test_data)
loss = loss_fn(y_test, test_labels)

pred = y_test.data.numpy()
label = test_labels.data.numpy()
pred_output = np.argmax(pred,axis=1)
no_errors = (pred_output != label)
no_errors =  no_errors.astype(int).sum()
ber_test = no_errors / 1500
ber_test = np.double(ber_test)

print("bit error rate : {}  loss : {}".format(ber_test, loss.item()))

bit error rate : 0.25866666666666666  loss : 0.40700143575668335
